In [2]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
plylst_data = pd.DataFrame(json_data)
plylst_data = plylst_data.drop(['plylst_title'], axis=1)
plylst_data.head()
plylst_data.shape

(115071, 5)

In [3]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
song_data = pd.DataFrame(json_data)
song_data.head()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4


In [4]:
with open('../Datasets/plylst_sample.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)
plylst_data = pd.DataFrame(json_data)
plylst_data.head()

,tags,id,songs,like_cnt,updt_date
99530,"[버스에서, 감성, 오후, 드라이브, 노을, 여행, 휴식, 기분전환, 퇴근길, 힐링]",36999,"[116776, 228329, 389448, 350353, 621643, 23548...",28,2020-04-23 13:18:21.000
95354,[팝],52669,"[230243, 523659, 125901, 529508, 679204, 35429...",0,2014-09-04 07:24:49.000
31136,[댄스],126460,"[534049, 252673, 430005, 168027, 643203, 14372...",1,2013-03-03 00:49:22.000
111189,[뉴에이지],98334,"[42027, 121050, 200689, 535869, 250279, 399064...",2,2014-07-02 18:35:36.000
54281,[댄스],148319,"[132031, 54755, 529748, 124632, 20973, 108207,...",10,2016-05-10 14:27:25.000


In [5]:
print(plylst_data.shape)

(11507, 5)


# 모델 방향성 수정
- 이전에 사용하던 협업 필터링을 사용한 모델은 현재 우리가 다루는 데이터에 적합하지 않음
- 따라서 모델 기반 필터링 중 Autoencoder 를 사용한 모델을 만들어 보기로 함

# Autoencoder (AE)
- Hidden Layer 가 하나인 뉴럴 네트워크
- 이 때, Input / Output data 가 같도록 네트워크를 학습
- Hidden Layer 에 추출된 의미 있는 데이터를 latent vector 라고도 함.

In [13]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from sklearn.preprocessing import OneHotEncoder

train_song = plylst_data['songs']
train_tag = plylst_data['tags']
song_list = [song for plist in train_song for song in plist ]
tag_list = [tag for plist in train_tag for tag in plist]
song_df = pd.DataFrame({'id':song_list})
#song_encoder = LabelEncoder()
#song_encoded = song_encoder.fit_transform(song_list)

tag_encoder = tf.one_hot(tag_list, len(tag_list))
tag_encoded = tag_encoder.fit_transform(tag_list.reshape(-1,1))
processed_data = np.concatenate((song_df[['id']], tag_encoded), axis=1)
precessed_data.head(20)

InvalidArgumentError: Value for attr 'TI' of string is not in the list of allowed values: uint8, int8, int32, int64
	; NodeDef: {{node OneHot}}; Op<name=OneHot; signature=indices:TI, depth:int32, on_value:T, off_value:T -> output:T; attr=axis:int,default=-1; attr=T:type; attr=TI:type,default=DT_INT64,allowed=[DT_UINT8, DT_INT8, DT_INT32, DT_INT64]> [Op:OneHot]